#Data Preparation

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 89.1/89.1 MB 1.1 MB/s eta 0:00:00


#Train

In [2]:
train=pd.read_csv('ArbuzTrain.csv')
train.head()

,client_id,average_check,number_orders,average_score,items_per_order,is_friend,last_check,last_score,last_scored_after_order_days,last_late_by_minutes,...,items_share_category_19,items_share_category_20,items_share_category_21,items_share_category_22,items_share_category_23,items_share_category_24,items_share_category_25,items_share_category_26,items_share_category_27,is_churn
0,301,49300,7,10.0,23.4286,0,9250.0,NaN,NaN,0,...,0.0,0.0,0.01,0.0,0.00,0.00,0.01,0.0,0.00,0
1,9394,58900,7,10.0,5.4286,0,24547.0,NaN,NaN,0,...,0.0,0.0,0.00,0.0,0.00,0.03,0.00,0.0,0.00,0
2,6680,43100,6,10.0,15.3333,0,6017.0,NaN,NaN,0,...,0.0,0.0,0.00,0.0,0.00,0.22,0.00,0.0,0.02,0
3,4774,33900,8,4.0,9.6250,0,7316.0,NaN,NaN,0,...,0.0,0.0,0.01,0.0,0.06,0.35,0.00,0.0,0.17,0
4,2722,37900,9,10.0,14.2222,0,2175.0,10.0,0.0,0,...,0.0,0.0,0.00,0.0,0.04,0.22,0.00,0.0,0.01,0


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10984 entries, 0 to 10983
Data columns (total 39 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   client_id                     10984 non-null  int64  
 1   average_check                 10984 non-null  int64  
 2   number_orders                 10984 non-null  int64  
 3   average_score                 9187 non-null   float64
 4   items_per_order               10984 non-null  float64
 5   is_friend                     10984 non-null  int64  
 6   last_check                    10979 non-null  float64
 7   last_score                    3788 non-null   float64
 8   last_scored_after_order_days  3788 non-null   float64
 9   last_late_by_minutes          10984 non-null  int64  
 10  is_last_order_fullfilled      10984 non-null  int64  
 11  items_share_category_1        10984 non-null  float64
 12  items_share_category_2        10984 non-null  float64
 13  i

In [4]:
train.isna().sum()

client_id                          0
average_check                      0
number_orders                      0
average_score                   1797
items_per_order                    0
is_friend                          0
last_check                         5
last_score                      7196
last_scored_after_order_days    7196
last_late_by_minutes               0
is_last_order_fullfilled           0
items_share_category_1             0
items_share_category_2             0
items_share_category_3             0
items_share_category_4             0
items_share_category_5             0
items_share_category_6             0
items_share_category_7             0
items_share_category_8             0
items_share_category_9             0
items_share_category_10            0
items_share_category_11            0
items_share_category_12            0
items_share_category_13            0
items_share_category_14            0
items_share_category_15            0
items_share_category_16            0
i

In [5]:
train.shape

(10984, 39)

In [6]:
train.isna().sum()

client_id                          0
average_check                      0
number_orders                      0
average_score                   1797
items_per_order                    0
is_friend                          0
last_check                         5
last_score                      7196
last_scored_after_order_days    7196
last_late_by_minutes               0
is_last_order_fullfilled           0
items_share_category_1             0
items_share_category_2             0
items_share_category_3             0
items_share_category_4             0
items_share_category_5             0
items_share_category_6             0
items_share_category_7             0
items_share_category_8             0
items_share_category_9             0
items_share_category_10            0
items_share_category_11            0
items_share_category_12            0
items_share_category_13            0
items_share_category_14            0
items_share_category_15            0
items_share_category_16            0
i

In [7]:
train['average_score'].value_counts()

10.0000    4565
8.0000      642
9.0000      346
9.3333      238
9.5000      214
           ... 
9.0625        1
9.6393        1
7.9412        1
9.9344        1
8.0870        1
Name: average_score, Length: 367, dtype: int64

In [8]:
train['satisfied']=np.where(train['average_score']>=6, 1, 0)
train['Frequent buyer']=np.where(train['number_orders']>=train['number_orders'].max()*0.8, 1, 0)
train['Total spendings']=train['number_orders']*train['average_check']
train['Average_order_item_price']=train['average_check']/train['items_per_order']
train['last_delivery_in_time']=np.where(train['last_late_by_minutes']>0, 0, 1)
train['last_order_delivery_level']=train['last_delivery_in_time'] + train['is_last_order_fullfilled']
train['Loyality level']=train['satisfied']+train['Frequent buyer']+train['is_friend']+train['last_order_delivery_level']
train['last_score']=train['last_score'].fillna(train['average_score'])

In [9]:
x_train=train.loc[:, ~train.columns.isin(['client_id','last_scored_after_order_days','is_churn'])]
x_train.shape

(10984, 43)

In [10]:
y_train=train.loc[:, train.columns == 'is_churn']
y_train.shape

(10984, 1)

#Imputation

In [11]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=3)
x_imputed=pd.DataFrame(imputer.fit_transform(x_train),columns=x_train.columns)
x_imputed['average_score'].isna().sum()

0

#Test

In [12]:
test=pd.read_csv('ArbuzTest.csv')
test.head()

,client_id,average_check,number_orders,average_score,items_per_order,is_friend,last_check,last_score,last_scored_after_order_days,last_late_by_minutes,...,items_share_category_18,items_share_category_19,items_share_category_20,items_share_category_21,items_share_category_22,items_share_category_23,items_share_category_24,items_share_category_25,items_share_category_26,items_share_category_27
0,5733,53500,9,10.0000,18.5556,1,9590.0,NaN,NaN,0,...,0.09,0.0,0.0,0.0,0.00,0.01,0.35,0.01,0.00,0.02
1,1559,41800,31,8.0000,11.3548,0,4839.0,8.0,12.0,0,...,0.06,0.0,0.0,0.0,0.01,0.02,0.38,0.01,0.01,0.03
2,8709,137900,12,10.0000,31.7500,0,33861.0,10.0,6.0,0,...,0.02,0.0,0.0,0.0,0.00,0.02,0.35,0.09,0.00,0.00
3,659,26700,27,10.0000,10.7407,0,8130.0,10.0,0.0,0,...,0.14,0.0,0.0,0.0,0.00,0.02,0.08,0.10,0.00,0.04
4,11662,31900,16,8.7273,8.8750,0,11428.0,10.0,2.0,0,...,0.17,0.0,0.0,0.0,0.00,0.02,0.24,0.01,0.00,0.00


In [13]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2747 entries, 0 to 2746
Data columns (total 38 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   client_id                     2747 non-null   int64  
 1   average_check                 2747 non-null   int64  
 2   number_orders                 2747 non-null   int64  
 3   average_score                 2285 non-null   float64
 4   items_per_order               2747 non-null   float64
 5   is_friend                     2747 non-null   int64  
 6   last_check                    2743 non-null   float64
 7   last_score                    950 non-null    float64
 8   last_scored_after_order_days  950 non-null    float64
 9   last_late_by_minutes          2747 non-null   int64  
 10  is_last_order_fullfilled      2747 non-null   int64  
 11  items_share_category_1        2747 non-null   float64
 12  items_share_category_2        2747 non-null   float64
 13  ite

In [14]:
test.isna().sum()

client_id                          0
average_check                      0
number_orders                      0
average_score                    462
items_per_order                    0
is_friend                          0
last_check                         4
last_score                      1797
last_scored_after_order_days    1797
last_late_by_minutes               0
is_last_order_fullfilled           0
items_share_category_1             0
items_share_category_2             0
items_share_category_3             0
items_share_category_4             0
items_share_category_5             0
items_share_category_6             0
items_share_category_7             0
items_share_category_8             0
items_share_category_9             0
items_share_category_10            0
items_share_category_11            0
items_share_category_12            0
items_share_category_13            0
items_share_category_14            0
items_share_category_15            0
items_share_category_16            0
i

In [15]:
test['satisfied']=np.where(test['average_score']>=6, 1, 0)
test['Frequent buyer']=np.where(test['number_orders']>=test['number_orders'].max()*0.8, 1, 0)
test['Total spendings']=test['number_orders']*test['average_check']
test['Average_order_item_price']=test['average_check']/test['items_per_order']
test['last_delivery_in_time']=np.where(test['last_late_by_minutes']>0, 0, 1)
test['last_order_delivery_level']=test['last_delivery_in_time'] + test['is_last_order_fullfilled']
test['Loyality level']=test['satisfied']+test['Frequent buyer']+test['is_friend']+test['last_order_delivery_level']
test['last_score']=test['last_score'].fillna(test['average_score'])

In [16]:
test.shape

(2747, 45)

In [17]:
x_test=test.loc[:, ~test.columns.isin(['client_id','last_scored_after_order_days'])]
x_test.shape

(2747, 43)

#Imputation

In [18]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=3)
x_test_imputed=pd.DataFrame(imputer.fit_transform(x_test),columns=x_test.columns)
x_test_imputed['average_score'].isna().sum()

0

XGBooster

In [19]:
from xgboost import XGBClassifier 
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import StratifiedKFold 
import matplotlib.pyplot  


In [20]:
model = XGBClassifier() 
n_estimators = range(50, 1000, 50) 
param_grid = dict(n_estimators=n_estimators) 
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7) 
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold) 
grid_result = grid_search.fit(x_imputed, y_train) 
 
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)) 
means = grid_result.cv_results_['mean_test_score'] 
stds = grid_result.cv_results_['std_test_score'] 
params = grid_result.cv_results_['params'] 
for mean, stdev, param in zip(means, stds, params): 
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.500858 using {'n_estimators': 50}
-0.500858 (0.010052) with: {'n_estimators': 50}
-0.525545 (0.010515) with: {'n_estimators': 100}
-0.549160 (0.012739) with: {'n_estimators': 150}
-0.570958 (0.015090) with: {'n_estimators': 200}
-0.594037 (0.018031) with: {'n_estimators': 250}
-0.616693 (0.019157) with: {'n_estimators': 300}
-0.639577 (0.020303) with: {'n_estimators': 350}
-0.658617 (0.022222) with: {'n_estimators': 400}
-0.677864 (0.023525) with: {'n_estimators': 450}
-0.695903 (0.026062) with: {'n_estimators': 500}
-0.714706 (0.029040) with: {'n_estimators': 550}
-0.731159 (0.030102) with: {'n_estimators': 600}
-0.746219 (0.031993) with: {'n_estimators': 650}
-0.760810 (0.033786) with: {'n_estimators': 700}
-0.774792 (0.034533) with: {'n_estimators': 750}
-0.786845 (0.035261) with: {'n_estimators': 800}
-0.799271 (0.036307) with: {'n_estimators': 850}
-0.810899 (0.036814) with: {'n_estimators': 900}
-0.820382 (0.037455) with: {'n_estimators': 950}


In [23]:
model = XGBClassifier() 
max_depth = range(1, 30, 2) 
param_grid = dict(max_depth=max_depth) 
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7) 
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold) 
grid_result = grid_search.fit(x_imputed, y_train) 
 
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)) 
means = grid_result.cv_results_['mean_test_score'] 
stds = grid_result.cv_results_['std_test_score'] 
params = grid_result.cv_results_['params'] 
for mean, stdev, param in zip(means, stds, params): 
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.472234 using {'max_depth': 1}
-0.472234 (0.007140) with: {'max_depth': 1}
-0.485583 (0.007019) with: {'max_depth': 3}
-0.501465 (0.009355) with: {'max_depth': 5}
-0.544113 (0.012464) with: {'max_depth': 7}
-0.589484 (0.017250) with: {'max_depth': 9}
-0.635195 (0.017988) with: {'max_depth': 11}
-0.654924 (0.021130) with: {'max_depth': 13}
-0.672727 (0.017678) with: {'max_depth': 15}
-0.688023 (0.022272) with: {'max_depth': 17}
-0.685847 (0.014495) with: {'max_depth': 19}
-0.684337 (0.018516) with: {'max_depth': 21}
-0.677051 (0.021073) with: {'max_depth': 23}
-0.686115 (0.015441) with: {'max_depth': 25}
-0.693207 (0.021340) with: {'max_depth': 27}
-0.682661 (0.010821) with: {'max_depth': 29}


In [31]:
model = XGBClassifier() 
min_child_weight = range(1, 20, 1) 
param_grid = dict(min_child_weight=min_child_weight) 
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7) 
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold) 
grid_result = grid_search.fit(x_imputed, y_train) 
 
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)) 
means = grid_result.cv_results_['mean_test_score'] 
stds = grid_result.cv_results_['std_test_score'] 
params = grid_result.cv_results_['params'] 
for mean, stdev, param in zip(means, stds, params): 
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.503601 using {'min_child_weight': 19}
-0.525545 (0.010515) with: {'min_child_weight': 1}
-0.525016 (0.015311) with: {'min_child_weight': 2}
-0.520797 (0.012641) with: {'min_child_weight': 3}
-0.519924 (0.011820) with: {'min_child_weight': 4}
-0.518611 (0.012616) with: {'min_child_weight': 5}
-0.516803 (0.015021) with: {'min_child_weight': 6}
-0.515345 (0.011643) with: {'min_child_weight': 7}
-0.511247 (0.012059) with: {'min_child_weight': 8}
-0.514849 (0.007694) with: {'min_child_weight': 9}
-0.511823 (0.008882) with: {'min_child_weight': 10}
-0.511467 (0.008870) with: {'min_child_weight': 11}
-0.509149 (0.012756) with: {'min_child_weight': 12}
-0.510868 (0.010913) with: {'min_child_weight': 13}
-0.509937 (0.013654) with: {'min_child_weight': 14}
-0.510033 (0.013807) with: {'min_child_weight': 15}
-0.506980 (0.009107) with: {'min_child_weight': 16}
-0.506818 (0.013369) with: {'min_child_weight': 17}
-0.507914 (0.009753) with: {'min_child_weight': 18}
-0.503601 (0.011061) with:

In [32]:
model = XGBClassifier() 
gamma = range(0,40,1) 
param_grid = dict(gamma=gamma) 
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7) 
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold) 
grid_result = grid_search.fit(x_imputed, y_train) 
 
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)) 
means = grid_result.cv_results_['mean_test_score'] 
stds = grid_result.cv_results_['std_test_score'] 
params = grid_result.cv_results_['params'] 
for mean, stdev, param in zip(means, stds, params): 
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.473549 using {'gamma': 11}
-0.525545 (0.010515) with: {'gamma': 0}
-0.521749 (0.010949) with: {'gamma': 1}
-0.505779 (0.015047) with: {'gamma': 2}
-0.496137 (0.013528) with: {'gamma': 3}
-0.488055 (0.010636) with: {'gamma': 4}
-0.484475 (0.012116) with: {'gamma': 5}
-0.480388 (0.010080) with: {'gamma': 6}
-0.479108 (0.010400) with: {'gamma': 7}
-0.475912 (0.008332) with: {'gamma': 8}
-0.475140 (0.007026) with: {'gamma': 9}
-0.475231 (0.006859) with: {'gamma': 10}
-0.473549 (0.006703) with: {'gamma': 11}
-0.475561 (0.006593) with: {'gamma': 12}
-0.475440 (0.006975) with: {'gamma': 13}
-0.475220 (0.006528) with: {'gamma': 14}
-0.475054 (0.006789) with: {'gamma': 15}
-0.475400 (0.006774) with: {'gamma': 16}
-0.475822 (0.006701) with: {'gamma': 17}
-0.475745 (0.006905) with: {'gamma': 18}
-0.475959 (0.007000) with: {'gamma': 19}
-0.476063 (0.006864) with: {'gamma': 20}
-0.475818 (0.006754) with: {'gamma': 21}
-0.476114 (0.006923) with: {'gamma': 22}
-0.476751 (0.006803) with: {'ga

In [33]:
from xgboost import XGBClassifier  
clas=XGBClassifier(learning_rate=1,  
 n_estimators=950, 
 max_depth=17,  
 min_child_weight=2,  
 gamma=2,  
 subsample=0.8,  
 colsample_bytree=0.8,  
 objective= 'binary:logistic',  
 scale_pos_weight=1)  
clas=clas.fit(x_imputed, y_train)  
pred=clas.predict(x_test_imputed)  
pred

array([0, 0, 0, ..., 0, 0, 0])

In [34]:
client_id=test['client_id']
np.array(client_id)
result=pd.DataFrame({'client_id':client_id,'is_churn':pred})
result

,client_id,is_churn
0,5733,0
1,1559,0
2,8709,0
3,659,0
4,11662,0
...,...,...
2742,948,0
2743,6983,1
2744,8864,0
2745,8877,0


In [38]:
result['is_churn'].value_counts()

0    2214
1     533
Name: is_churn, dtype: int64

In [39]:
result.to_csv('result_xg_final.csv', index=False)